# Block Detail
In this notebook, I extract block details given a start_block number and end_block number. Information extracted include:
- block detail
- gas usage
- timestamp
- sibling blocks

In [1]:
import os
from hexbytes import HexBytes
import csv
import numpy as np
import sys
import copy
import web3_api

In [2]:
block_start = 13500000
block_end = 15850000
database = {} # store transaction hash with timestamp
blockdetail = {} # store block details